In [3]:
import requests
import sqlite3
import sys
sys.path.append("..")
from rcn_py import orcid

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [40]:
def create_rsd_database():
    con = sqlite3.connect("rsd.db")
    cur = con.cursor()
    cur.execute("CREATE TABLE project (id primary key, title, description)")
    cur.execute("CREATE TABLE author(id primary key, project, orcid, given_name, family_name, affiliation, FOREIGN KEY(project) REFERENCES project(id))")
    cur.execute("CREATE TABLE software(id primary key, doi, brand_name, description)")
    cur.execute("CREATE TABLE contributor(id primary key, software, orcid, given_name, family_name, affiliation, FOREIGN KEY(software) REFERENCES software(id))")
    con.commit()
    con.close()
    return "Created"


In [41]:
create_rsd_database()

'Created'

In [4]:
def store_rsd():
    response_project = requests.get("https://research-software-directory.org/api/v1/project?select=id,title,description")
    response_author = requests.get("https://research-software-directory.org/api/v1/team_member?select=id,project,orcid,given_names,family_names, affiliation")
    response_software = requests.get("https://research-software-directory.org/api/v1/software?select=id,concept_doi,brand_name, description")
    response_contributor = requests.get("https://research-software-directory.org/api/v1/contributor?select=id,software,orcid,given_names,family_names,affiliation")
    projects = response_project.json()
    authors_proj = response_author.json()
    software = response_software.json()
    contributor_soft = response_contributor.json()

    con = sqlite3.connect("rsd.db")
    cur = con.cursor()
    for p in projects:
        # con = sqlite3.connect("rsd.db")
        # cur = con.cursor()
        cur.execute("INSERT or IGNORE INTO project VALUES (?, ?, ?)", (p['id'], p['title'], p['description']))
        # con.commit()
        # con.close()

    
    for a in authors_proj:
        # con = sqlite3.connect("rsd.db")
        # cur = con.cursor()
        if a['orcid'] == None:
            a['orcid'] = orcid.name_to_orcid_id(a['given_names'] + ' ' + a['family_names'])
        cur.execute("INSERT or IGNORE INTO author VALUES (?, ?, ?, ?, ?, ?)", (a['id'], a['project'], a['orcid'], a['given_names'], a['family_names'], a['affiliation']))
        # con.commit()
        # con.close()

    
    for s in software:
        # con = sqlite3.connect("rsd.db")
        # cur = con.cursor()
        cur.execute("INSERT or IGNORE INTO software VALUES (?, ?, ?, ?)", (s['id'], s['doi'], s['brand_name'], p['description']))
        # con.commit()
        # con.close()

    
    for c in contributor_soft:
        # con = sqlite3.connect("rsd.db")
        # cur = con.cursor()
        if c['orcid'] == None:
            c['orcid'] = orcid.name_to_orcid_id(c['given_names'] + ' ' + c['family_names'])
        cur.execute("INSERT or IGNORE INTO author VALUES (?, ?, ?, ?, ?, ?)", (c['id'], c['software'], c['orcid'], c['given_names'], c['family_names'], c['affiliation']))
    con.commit()
    con.close()
    return "Done"



In [5]:
store_rsd()

KeyboardInterrupt: 

In [5]:
response2 = requests.get("https://research-software-directory.org/api/v1/team_member?select=id,project,orcid,given_names,family_names")
temp2 = response2.json()

In [45]:
if temp2[0]['orcid'] ==None:
    temp2[0]['orcid'] = orcid.name_to_orcid_id(temp2[0]['given_names'] + ' ' + temp2[0]['family_names'])

In [46]:
temp2

[{'id': 'db1d71e8-4a3b-457a-b5d6-325ae5bce9be',
  'project': '2a94df7c-deb2-4278-b975-506f2c8a64f8',
  'orcid': '0000-0002-2662-1994',
  'given_names': 'Dafne',
  'family_names': 'van Kuppevelt'},
 {'id': 'a895fad3-f21e-4572-ba32-887f602e5204',
  'project': '0c4cdce5-6ee4-4a21-8189-3f9119190b9d',
  'orcid': None,
  'given_names': 'A.L.A.J.',
  'family_names': 'Dekker'},
 {'id': 'c03ad5e7-7094-413d-8d6d-67975650b171',
  'project': '0c4cdce5-6ee4-4a21-8189-3f9119190b9d',
  'orcid': None,
  'given_names': 'Djura',
  'family_names': 'Smits'},
 {'id': '067caf62-45b8-43e4-99a0-83d225138eea',
  'project': 'eb57dee4-3f43-403b-bf03-84558b81e05e',
  'orcid': '0000-0003-4753-4133',
  'given_names': 'Juan E.',
  'family_names': 'Machado'},
 {'id': 'b5e61c04-145a-4884-896f-a02f8a3dcc0a',
  'project': '80343f6c-ab3f-4c51-bc53-5b954fb2dd8f',
  'orcid': '0000-0002-1263-1067',
  'given_names': 'Fred',
  'family_names': 'Wubs'},
 {'id': '7d64d690-d6df-4cd8-9d1e-0eb05eed8112',
  'project': 'eb57dee4-3f43

In [22]:
response3 = requests.get("https://research-software-directory.org/api/v1/software?select=id,concept_doi,description")
temp3 = response3.json()

In [23]:
temp3

[{'id': '77201671-58bf-4d6e-9f3d-d509c0ff5b88',
  'concept_doi': '10.5281/zenodo.597262',
  'description': '* For cheminformaticans who want to do structure-based protein binding site comparison and bioisosteric replacement for ligand design\n* It makes the Kripo Python library available in the KNIME workflow platform as workflow nodes.\n* Kripo encodes the interactions of protein and bound ligand also known as a pharmacophore into a fingerprint, the fingerprints can be compared to each other to find similar pharmacophores\n* The Kripo software is open source while most other similar software is commercial or requires registration'},
 {'id': 'b8680d8a-b946-4d5a-8fc9-1c86184c4824',
  'concept_doi': '10.5281/zenodo.997332',
  'description': '* For cheminformations who are working with pharmacphores in KNIME workflows.\n* Adds pharmacophore data type to KNIME\n* Adds nodes to read, write and align pharmacophores\n\n\nA pharmacophore is an abstract description of molecular features that ar

In [24]:
response4 = requests.get("https://research-software-directory.org/api/v1/contributor?select=id,software,orcid,affiliation")
temp4 = response4.json()

In [25]:
temp4

[{'id': '4e866d63-d9da-491c-a81b-ff5faef2584c',
  'software': '4f076d31-3763-43ed-a959-3ca9ff9e2c6a',
  'orcid': '0000-0003-0182-9008',
  'affiliation': 'Accelting'},
 {'id': 'd91b3514-a327-4980-ac20-1b14ce135168',
  'software': '25ac7630-b959-4446-a484-84ebab43232b',
  'orcid': None,
  'affiliation': 'University of Amsterdam, Netherlands eScience Center'},
 {'id': '81c6afb0-79e1-4059-88c4-27eeea36bfd9',
  'software': '25ac7630-b959-4446-a484-84ebab43232b',
  'orcid': '0000-0002-7508-3272',
  'affiliation': 'Netherlands eScience Center'},
 {'id': 'ee641a4b-7007-42a8-bbd7-e19641d6fdff',
  'software': '51247fea-11a0-497e-b658-bd47d9391618',
  'orcid': '0000-0002-7299-7992',
  'affiliation': 'Tilburg University'},
 {'id': '02685e00-8148-42ae-bdf6-fd0bf5a22e86',
  'software': '21d13da1-afac-40a5-9ba6-96323156cd03',
  'orcid': None,
  'affiliation': 'VU University Amsterdam'},
 {'id': 'cb4b525c-3517-4c59-8d20-fbea5bc648fe',
  'software': '153f4ae9-a031-4525-a257-908dbddca5ef',
  'orcid': '0